In [1]:
# pip install psycopg2
# pip install psycopg2-binary
# conda install -c anaconda psycopg2

In [15]:
import psycopg2
import pandas as pd

In [16]:
conn = psycopg2.connect(database = "dev", 
                        user = "admin", 
                        host= 'redshift-workgroup.604238822057.us-east-1.redshift-serverless.amazonaws.com',
                        password = "Sf!323895474",
                        port = '5439')

In [17]:
cur = conn.cursor()

In [30]:
cur.exeute('''DROP TABLE IF EXISTS log_data;
CREATE TABLE log_data (
  artist NVARCHAR(255),
  auth NVARCHAR(75),
  firstName NVARCHAR(150),
  gender VARCHAR(1),
  itemInSession INT,
  lastName NVARCHAR(150),
  length DECIMAL(20, 10),
  level NVARCHAR(150),
  location NVARCHAR(300),
  method NVARCHAR(75),
  page NVARCHAR(75),
  registration DECIMAL(25, 1),
  sessionID INT,
  song NVARCHAR(250),
  status INT,
  ts BIGINT,
  userAgent NVARCHAR(300),
  userID INT
);''')

In [31]:
conn.commit()

In [44]:
cur.excute('''DROP TABLE IF EXISTS song_data;
CREATE TABLE song_data (
  artist_id NVARCHAR(150),
  artist_latitude DECIMAL(30,15),
  artist_location NVARCHAR(250),
  artist_longitude DECIMAL(30,15),
  artist_name NVARCHAR(350),
  duration DECIMAL(15, 7),
  num_songs INT,
  song_id NVARCHAR(150),
  title NVARCHAR(250),
  year INT
);''')

In [45]:
conn.commit()

In [55]:
copy_logs = '''copy log_data
from 's3://udacity-dend/log_data'
iam_role 'arn:aws:iam::604238822057:role/MyRedshiftRole' 
region 'us-west-2'
json 'auto ignorecase';'''

cur.execute(copy_logs)

In [56]:
conn.commit()

In [58]:
logs = pd.read_sql_query('select * from log_data', conn)
logs

C:\Users\The user\AppData\Local\Temp\ipykernel_21832\1804257470.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  logs = pd.read_sql_query('select * from log_data', conn)


,artist,auth,firstname,gender,iteminsession,lastname,length,level,location,method,page,registration,sessionid,song,status,ts,useragent,userid
0,None,Logged In,Jayden,M,22,Graves,NaN,paid,"Marinette, WI-MI",GET,Help,1.540664e+12,231,None,200,1541444403796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25.0
1,Linkin Park,Logged In,Jayden,M,23,Graves,289.90649,paid,"Marinette, WI-MI",PUT,NextSong,1.540664e+12,231,Shadow Of The Day (Album Version),200,1541444568796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25.0
2,Vampire Weekend,Logged In,Jayden,M,0,Fox,137.82159,free,"New Orleans-Metairie, LA",PUT,NextSong,1.541034e+12,296,A-Punk (Album),200,1541451499796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101.0
3,Crystal,Logged In,Jayden,M,1,Fox,251.97669,free,"New Orleans-Metairie, LA",PUT,NextSong,1.541034e+12,296,VÃÂNDORMADÃÂR,200,1541451636796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101.0
4,Plain White T S,Logged In,Theodore,M,0,Smith,266.94485,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540306e+12,261,A Lonely September (Album),200,1541453165796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8051,The Strokes,Logged In,Avery,F,16,Watkins,187.34975,paid,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.540872e+12,884,You Only Live Once,200,1543557890796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30.0
8052,Animals As Leaders,Logged In,Chloe,F,18,Cuevas,323.83955,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,1079,Tempting Time,200,1543557895796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49.0
8053,Mazzy Star,Logged In,Avery,F,17,Watkins,297.95220,paid,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.540872e+12,884,Flowers In December,200,1543558077796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30.0
8054,Purple Ribbon All-Stars feat. Big Boi,Logged In,Chloe,F,21,Cuevas,242.83383,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,1041,Kryptonite (Radio Edit) (feat. Big Boi),200,1543504958796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49.0


In [54]:
copy_songs = '''copy song_data
from 's3://udacity-dend/song_data'
iam_role 'arn:aws:iam::604238822057:role/MyRedshiftRole' 
region 'us-west-2'
json 'auto ignorecase';'''

cur.execute(copy_songs)

In [ ]:
conn.commit()

In [109]:
songs = pd.read_sql_query('select * from song_data', conn)

C:\Users\The user\AppData\Local\Temp\ipykernel_21832\3293062927.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  songs = pd.read_sql_query('select * from song_data', conn)


In [151]:
conn.commit()

In [137]:
# Fact Table
# songplays - records in event data associated with song plays i.e. records with page NextSong
create_ftable_songplays = '''DROP TABLE IF EXISTS songplays;
CREATE TABLE songplays (
  songplays_id bigint primary key generated by default as identity,
  start_time TIMESTAMP,
  user_id INT,
  level NVARCHAR(150),
  song_id NVARCHAR(150),
  artist_id NVARCHAR(150),
  session_id INT,
  location NVARCHAR(300),
  user_agent NVARCHAR(300)
);'''

cur.execute(create_ftable_songplays)

In [139]:
# Dimension Tables
# users - users in the app
# user_id, first_name, last_name, gender, level
create_dtable_users = '''DROP TABLE IF EXISTS users;
CREATE TABLE users (
  user_id INT PRIMARY KEY,
  first_name NVARCHAR(150),
  last_name NVARCHAR(150),
  gender VARCHAR(1),
  level NVARCHAR(150)
);'''

cur.execute(create_dtable_users)

# songs - songs in music database
# song_id, title, artist_id, year, duration
create_dtable_songs = '''DROP TABLE IF EXISTS songs;
CREATE TABLE songs (
  song_id NVARCHAR(150) PRIMARY KEY,
  title NVARCHAR(250),
  artist_id NVARCHAR(150),
  year INT,
  duration DECIMAL(15, 7)
);'''

cur.execute(create_dtable_songs)

# artists - artists in music database
# artist_id, name, location, latitude, longitude
create_dtable_artists = '''DROP TABLE IF EXISTS artists;
CREATE TABLE artists (
  artist_id NVARCHAR(150) PRIMARY KEY,
  name NVARCHAR(350),
  location NVARCHAR(250),
  latitude DECIMAL(30,15),
  longitude DECIMAL(30,15)
);'''

cur.execute(create_dtable_artists)


# time - timestamps of records in songplays broken down into specific units
# start_time, hour, day, week, month, year, weekday
create_dtable_time = '''DROP TABLE IF EXISTS time;
CREATE TABLE time (
  start_time TIMESTAMP PRIMARY KEY,
  hour VARCHAR(2),
  day VARCHAR(2),
  week VARCHAR(2),
  month VARCHAR(2),
  year VARCHAR(4),
  weekday VARCHAR(1)
);'''

cur.execute(create_dtable_time)

In [149]:
insert_songplays = '''INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) 
(SELECT t.start_time, 
u.user_id,
u.level,
s.song_id,
a.artist_id,
l.sessionID,
l.location,
l.userAgent
FROM log_data l
JOIN users u
ON l.userID = u.user_id
LEFT JOIN song_data sd
ON sd.artist_name = l.artist
JOIN songs s
ON s.song_id = sd.song_id
JOIN artists a
ON a.artist_id = sd.artist_id
JOIN time t
ON t.start_time = timestamp 'epoch' + CAST(l.ts AS BIGINT)/1000 * interval '1 second')'''
cur.execute(insert_songplays)

In [141]:
insert_users = '''INSERT INTO users (user_id, first_name, last_name, gender, level)
(SELECT DISTINCT
    userID,
    firstName,
    lastName,
    gender,
    level
FROM
    log_data
    WHERE userID IS NOT NULL);'''
cur.execute(insert_users)

In [142]:
insert_songs = '''INSERT INTO songs (song_id, title, artist_id, year, duration)
(SELECT DISTINCT
    song_id,
    title,
    artist_id,
    year,
    duration
FROM
    song_data);'''
cur.execute(insert_songs)

In [143]:
insert_artists = '''INSERT INTO artists (artist_id, name, location, latitude, longitude)
(SELECT DISTINCT
    artist_id,
    artist_name,
    artist_location,
    artist_latitude,
    artist_longitude
FROM
    song_data);'''
cur.execute(insert_artists)

In [144]:
insert_time = '''INSERT INTO time (start_time, hour, day, week, month, year, weekday)
(SELECT DISTINCT
    timestamp 'epoch' + CAST(ts AS BIGINT)/1000 * interval '1 second',
    EXTRACT (HOUR FROM timestamp 'epoch' + CAST(ts AS BIGINT)/1000 * interval '1 second'),
    EXTRACT (DAY FROM timestamp 'epoch' + CAST(ts AS BIGINT)/1000 * interval '1 second'),
    EXTRACT (WEEK FROM timestamp 'epoch' + CAST(ts AS BIGINT)/1000 * interval '1 second'),
    EXTRACT (MONTH FROM timestamp 'epoch' + CAST(ts AS BIGINT)/1000 * interval '1 second'),
    EXTRACT (YEAR FROM timestamp 'epoch' + CAST(ts AS BIGINT)/1000 * interval '1 second'),
    EXTRACT (WEEKDAY FROM timestamp 'epoch' + CAST(ts AS BIGINT)/1000 * interval '1 second')
FROM
    log_data);'''
cur.execute(insert_time)

In [148]:
conn.rollback()

In [150]:
cur.execute("select * from songplays")
cur.fetchall()

[(12291,
  datetime.datetime(2018, 11, 6, 21, 51, 14),
  97,
  'paid',
  'SOCPJYN12A81C21742',
  'AR8ICRA1187B9B8A5F',
  293,
  'Lansing-East Lansing, MI',
  '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"'),
 (23187,
  datetime.datetime(2018, 11, 28, 9, 6, 27),
  58,
  'paid',
  'SOSHUVD12A6701F8F9',
  'ARJ7KF01187B98D717',
  887,
  'Augusta-Richmond County, GA-SC',
  '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"'),
 (23827,
  datetime.datetime(2018, 11, 21, 2, 16, 18),
  97,
  'paid',
  'SOSHUVD12A6701F8F9',
  'ARJ7KF01187B98D717',
  671,
  'Lansing-East Lansing, MI',
  '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"'),
 (933,
  datetime.datetime(2018, 11, 29, 21, 24, 35),
  49,
  'paid',
  'SOVBTZQ12AB0184DC0',
  'AREKO1L1187B997EFE',
  1041,
  'San Francisco-Oakland-Hayward, CA',
  'Mozilla/5.0 (Wi

In [ ]:
cur.exute('''DROP TABLE IF EXISTS log_data;
CREATE TABLE log_data (
  artist NVARCHAR(255),
  auth NVARCHAR(75),
  firstName NVARCHAR(150),
  gender VARCHAR(1),
  itemInSession INT,
  lastName NVARCHAR(150),
  length DECIMAL(20, 10),
  level NVARCHAR(150),
  location NVARCHAR(300),
  method NVARCHAR(75),
  page NVARCHAR(75),
  registration DECIMAL(25, 1),
  sessionID INT,
  song NVARCHAR(250),
  status INT,
  ts BIGINT,
  userAgent NVARCHAR(300),
  userID INT
);''')
cur.exete('''DROP TABLE IF EXISTS song_data;
CREATE TABLE song_data (
  artist_id NVARCHAR(150),
  artist_latitude DECIMAL(30,15),
  artist_location NVARCHAR(250),
  artist_longitude DECIMAL(30,15),
  artist_name NVARCHAR(350),
  duration DECIMAL(15, 7),
  num_songs INT,
  song_id NVARCHAR(150),
  title NVARCHAR(250),
  year INT
);''')

In [152]:
cur.close()
conn.close()

In [5]:
cur.execute('''select sd.*, l.* from song_data sd
join log_data l
on l.song = sd.title and l.artist = sd.artist_name''')

In [22]:
pd.read_sql_query('''SELECT  TIMESTAMP 'epoch' + l.ts/1000 * INTERVAL '1 second',
l.userId,
l.level,
s.song_id,
s.artist_id,
s.artist_name,
l.sessionID,
l.location,
l.userAgent
FROM log_data l
JOIN song_data s
ON s.artist_name = l.artist AND l.song = s.title AND s.duration >= l.length
where l.page = 'NextSong';''', conn)

C:\Users\The user\AppData\Local\Temp\ipykernel_6248\461181678.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''SELECT  TIMESTAMP 'epoch' + l.ts/1000 * INTERVAL '1 second',


,?column?,userid,level,song_id,artist_id,artist_name,sessionid,location,useragent
0,2018-11-05 17:00:27,73,paid,SOBANHD12A58A7BB7C,ARSUFX91187FB3B73E,Pillar,255,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
1,2018-11-14 15:47:47,80,paid,SOHOVIP12A6D4F9267,ARQQ5B61187B9B4F61,Sparklehorse,574,"Portland-South Portland, ME","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
2,2018-11-13 11:46:10,50,free,SOBJDDA12A6BD53159,ARCS4GZ1187FB469EB,Slim Dusty,430,"New Haven-Milford, CT","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
3,2018-11-06 08:49:19,12,free,SOBJDDA12A6BD53159,ARCS4GZ1187FB469EB,Slim Dusty,300,"New York-Newark-Jersey City, NY-NJ-PA",Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20...
4,2018-11-08 14:58:28,29,paid,SOIZLKI12A6D4F7B61,ARR3ONV1187B9A2F59,Muse,372,"Atlanta-Sandy Springs-Roswell, GA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
...,...,...,...,...,...,...,...,...,...
248,2018-11-04 07:31:31,25,paid,SOHWVJJ12AB0185F6D,ARASYMJ1187B9ACAF2,Whitesnake,128,"Marinette, WI-MI","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
249,2018-11-09 17:55:00,80,paid,SOAOJYY12A58A7B2F9,ARFVYJI1187B9B8E13,Pearl Jam,416,"Portland-South Portland, ME","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
250,2018-11-28 07:11:30,58,paid,SOTSQDK12AB0184D15,ARPTKAK1187FB5308A,Plus One,887,"Augusta-Richmond County, GA-SC","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
251,2018-11-07 01:42:43,8,free,SOWTZNU12AB017EADB,AR6NYHH1187B9BA128,Yeah Yeah Yeahs,181,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."


In [23]:
pd.read_sql_query('''SELECT * from songplays;''', conn)

C:\Users\The user\AppData\Local\Temp\ipykernel_6248\1347120737.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''SELECT * from songplays;''', conn)


,songplays_id,start_time,user_id,level,song_id,artist_id,artist_name,session_id,location,user_agent
0,327,2018-11-04 09:41:55,44,paid,SOAFQGA12A8C1367FA,AR0IVTL1187B9AD520,Katy Perry,196,"Waterloo-Cedar Falls, IA",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...
1,106,2018-11-10 09:29:42,44,paid,SOWLLXC12AB0180FFE,AR66PLO1187FB4C8E5,The Audition,350,"Waterloo-Cedar Falls, IA",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...
2,734,2018-11-04 15:39:36,78,free,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,Dwight Yoakam,176,"Indianapolis-Carmel-Anderson, IN",Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20...
3,6,2018-11-05 11:08:56,95,paid,SOQYHVZ12A6D4F93CF,ARRZUPG11F43A69EF7,Jackson 5,222,"Winston-Salem, NC","""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like..."
4,137,2018-11-08 14:58:28,29,paid,SOIZLKI12A6D4F7B61,ARR3ONV1187B9A2F59,Muse,372,"Atlanta-Sandy Springs-Roswell, GA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
...,...,...,...,...,...,...,...,...,...,...
248,32,2018-11-22 13:21:21,63,free,SOLWQQJ12A8C13A2BD,ARDOL0J1187B9AC52A,Hooligans,785,"Santa Rosa, CA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
249,312,2018-11-28 13:45:00,97,paid,SOUNZHU12A8AE47481,AR37SX11187FB3E164,Ron Carter,944,"Lansing-East Lansing, MI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
250,250,2018-11-28 18:36:57,88,paid,SONHWUN12AC468C014,ARJIG5P1187B98D9DE,Fisher,999,"Sacramento--Roseville--Arden-Arcade, CA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
251,199,2018-11-09 19:57:57,36,paid,SODWXQV12A6310F10D,AR6892W1187B9AC71B,Placebo,392,"Janesville-Beloit, WI","""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537...."
